In [1]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.4.1
torchvision version: 0.19.1


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [18]:
import os
import shutil
from sklearn.model_selection import train_test_split

dataset_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'
directories = ['processed_lsd_jpgs', 'processed_osf_jpgs', 'processed_spider_jpgs', 'processed_tseg_jpgs']

train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

for d in directories:
    os.makedirs(os.path.join(train_dir, d), exist_ok=True)
    os.makedirs(os.path.join(test_dir, d), exist_ok=True)

for d in directories:
    image_dir = os.path.join(dataset_path, d)
    images = os.listdir(image_dir)

    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(train_dir, d, img)  
        shutil.copy(src_path, dest_path)

    for img in test_images:
        src_path = os.path.join(image_dir, img)
        dest_path = os.path.join(test_dir, d, img) 
        shutil.copy(src_path, dest_path)

print("Train-test split completed.")


Train-test split completed.


In [19]:
data_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data'


In [20]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [21]:
train_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/train'
test_dir = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/test'

In [22]:
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int=NUM_WORKERS):

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  class_names = train_data.classes

  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False,
      num_workers=num_workers,
      pin_memory=True,
  )
  return train_datalaoder, test_dataloader, class_names


In [23]:
manual_transforms = transforms.Compose([
    transforms.Resize([224,224]),
    transforms.ToTensor(),
    normalize
]) # can also use automatic transforms using weights.transforms()

train_dataloaders, test_dataloaders, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=manual_transforms,
    batch_size=32
)

train_dataloaders, test_dataloaders, class_names

(<torch.utils.data.dataloader.DataLoader at 0x315bfcb10>,
 ['processed_lsd_jpgs',
  'processed_osf_jpgs',
  'processed_spider_jpgs',
  'processed_tseg_jpgs'])

In [24]:
class_names

['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [25]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

In [26]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)


In [27]:
for param in model.features.parameters():
    param.requires_grad = False
    

set_seeds() 

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

In [28]:
from torchinfo import summary


In [29]:
summary(model, 
        input_size=(32, 3, 224, 224), 
        verbose=0,
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 4]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
try:
    from torch.utils.tensorboard import SummaryWriter
except:
    print("[INFO] Couldn't find tensorboard... installing it.")
    !pip install -q tensorboard
    from torch.utils.tensorboard import SummaryWriter


writer = SummaryWriter()

In [46]:
from typing import Dict, List
from tqdm.auto import tqdm

from going_modular.going_modular.engine import train_step, test_step


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,
                                           device=device)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,
                                        device=device)

        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    
        writer.add_scalars(main_tag="Loss", 
                           tag_scalar_dict={"train_loss": train_loss,
                                            "test_loss": test_loss},
                           global_step=epoch)

        writer.add_scalars(main_tag="Accuracy", 
                           tag_scalar_dict={"train_acc": train_acc,
                                            "test_acc": test_acc}, 
                           global_step=epoch)
        
        writer.add_graph(model=model, 
                         input_to_model=torch.randn(32, 3, 224, 224).to(device))
    
    writer.close()
    
    return results

In [47]:
set_seeds()
results = train(model=model,
                train_dataloader=train_dataloaders,
                test_dataloader=test_dataloaders,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=5,
                device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6684 | train_acc: 0.8233 | test_loss: 0.5110 | test_acc: 0.8620


 20%|██        | 1/5 [03:06<12:26, 186.56s/it]

Epoch: 2 | train_loss: 0.2865 | train_acc: 0.9171 | test_loss: 0.2999 | test_acc: 0.9102


 40%|████      | 2/5 [06:21<09:35, 191.71s/it]

...

 60%|██████    | 3/5 [09:41<06:30, 195.43s/it]

Epoch: 4 | train_loss: 0.1848 | train_acc: 0.9533 | test_loss: 0.1851 | test_acc: 0.9375


 80%|████████  | 4/5 [12:54<03:14, 194.30s/it]

Epoch: 5 | train_loss: 0.1654 | train_acc: 0.9485 | test_loss: 0.1683 | test_acc: 0.9336


100%|██████████| 5/5 [16:06<00:00, 193.38s/it]


In [48]:
results

{'train_loss': [0.6683949856988846,
  0.28645355446684745,
  0.2099980918630477,
  0.1847972752105805,
  0.16537261345694143],
 'train_acc': [0.8232758620689655,
  0.9171301446051168,
  0.9340586763070078,
  0.9533161846496108,
  0.948484427141268],
 'test_loss': [0.5109908431768417,
  0.2999418741092086,
  0.21004890464246273,
  0.1850558731239289,
  0.16827066976111382],
 'test_acc': [0.8619791666666666, 0.91015625, 0.9296875, 0.9375, 0.93359375]}

In [49]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 14292), started 0:39:56 ago. (Use '!kill 14292' to kill it.)

In [1]:
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

[INFO] torch/torchvision versions not as required, installing nightly versions.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
torch version: 2.4.1
torchvision version: 0.19.1


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

try:
    from going_modular.going_modular import data_setup, engine
except:
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[.229, 0.224, 0.225])

In [6]:
class_names = ['processed_lsd_jpgs',
 'processed_osf_jpgs',
 'processed_spider_jpgs',
 'processed_tseg_jpgs']

In [7]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

In [8]:
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
for param in model.features.parameters():
    param.requires_grad = False

model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, 
              out_features=len(class_names),
              bias=True).to(device))

model.load_state_dict(torch.load('model_final_weights.pth'))


/var/folders/4x/9705y5ws5p31tnxd9bmg3ll00000gn/T/ipykernel_27990/1418258017.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_fina

<All keys matched successfully>

In [9]:
preprocess = transforms.Compose([
    transforms.Resize(256),    
    transforms.CenterCrop(224),  
    transforms.ToTensor(),      
    transforms.Normalize(        
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])


In [10]:
from PIL import Image
img_path = '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_spider_jpgs/1_t2.jpg'  # Replace with your image path
image = Image.open(img_path).convert('RGB')
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  


In [11]:
input_batch = input_batch.to(device)


In [12]:
model.eval()

with torch.no_grad():
    output = model(input_batch)  
_, predicted_idx = torch.max(output, 1)


In [13]:
predicted_class = class_names[predicted_idx.item()]

print(f'Predicted class: {predicted_class}')


Predicted class: processed_spider_jpgs


In [14]:
from PIL import Image
import torch
from torchvision import transforms

def predict_image_class(image_path, model, class_names, device='cpu'):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)
    
    input_batch = input_batch.to(device)
    model.eval()
    
    with torch.no_grad():
        output = model(input_batch)
        _, predicted_idx = torch.max(output, 1)
    
    predicted_class = class_names[predicted_idx.item()]
    return predicted_class


In [15]:
predict_image_class('/Users/ishmeetsinghnagi/CODES/Lumbar Spine/data/processed_spider_jpgs/1_t2.jpg', model, class_names, device='cpu')

'processed_spider_jpgs'

In [18]:
from PIL import Image
import torch
from torchvision import transforms
import torch.nn.functional as F

def predict_image_class2(image_path, model, class_names, device='cpu', confidence_threshold=50):
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image = Image.open(image_path).convert('RGB')
    tensor = preprocess(image)
    batch = tensor.unsqueeze(0)
    
    batch = batch.to(device)
    model.eval()
    
    with torch.no_grad():
        output = model(batch)
        probabilities = F.softmax(output, dim=1)
        
        _, predicted_idx = torch.max(output, 1)
        confidence = probabilities[0][predicted_idx].item() * 100  
    
    if confidence < confidence_threshold:
        return "Not anythin related to Spines",confidence
    else:
        predicted_class = class_names[predicted_idx.item()]
        return predicted_class, confidence


In [19]:
predict_image_class2('/Users/ishmeetsinghnagi/CODES/Lumbar Spine/IMG_7774.jpg', model, class_names, device='cpu')

UnidentifiedImageError: cannot identify image file '/Users/ishmeetsinghnagi/CODES/Lumbar Spine/IMG_7774.jpg'